In [7]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from matplotlib.ticker import FormatStrFormatter

from sklearn.preprocessing import StandardScaler
import sklearn
import pickle

# from utils import plot_projections

from scipy.stats import kurtosis, skew

%matplotlib widget

In [8]:
def extract_speaker_num(f):
    
    substr = f.split('/')[7].split('_')
    
    speaker_str = re.split('(\d+)', substr[2])
    
    return int(speaker_str[1])

In [9]:
# train_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/train_text_independent/'
train_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/train/'
train_files_laser = list(glob.glob(os.path.join(train_dir + 'laser/', '*.wav')))
train_files_laser.sort()
print(train_files_laser)
print(len(train_files_laser))

['/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker10_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker10_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker10_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker10_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker10_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker11_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker11_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker11_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker11_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker11_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker12_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker12_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/laser/speaker12_003.wav'

In [10]:
train_files_original = list(glob.glob(os.path.join(train_dir + 'original/', '*.wav')))
train_files_original.sort()
print(train_files_original)
print(len(train_files_original))

['/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker10_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker10_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker10_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker10_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker10_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker11_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker11_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker11_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker11_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker11_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker12_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/train/original/speaker12_002.wav', '/home/hashim/PHD/audio_data/AllAudioSa

In [11]:
# test_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/test_text_independent/'
test_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/test/'

In [12]:
test_files_laser = list(glob.glob(os.path.join(test_dir + 'laser/', '*.wav')))
test_files_laser.sort()
print(test_files_laser)
print(len(test_files_laser))

['/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker15_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker15_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker15_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker15_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker15_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker16_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker16_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker16_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker16_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker16_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker17_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker17_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/laser/speaker17_003.wav', '/home/hash

In [13]:
test_files_original = list(glob.glob(os.path.join(test_dir + 'original/', '*.wav')))
test_files_original.sort()
print(test_files_original)
print(len(test_files_original))

['/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker15_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker15_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker15_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker15_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker15_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker16_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker16_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker16_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker16_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker16_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker17_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/original/speaker17_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/test/o

### Computing Coefficients

In [14]:
def compute_coefficients(files_ls):
    
    db1 = pywt.Wavelet('db1')
    
    sigma_feat_ls = []
    skew_feat_ls = []
    kurt_feat_ls = []

    for af in files_ls:
        audio_data, sr = librosa.load(af, res_type='kaiser_fast')
        audio_data = librosa.util.normalize(audio_data)
        
        coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)


        sigma_feat = []
        skew_feat = []
        kurt_feat = []

        for cf in coeffs:

#             sigma_feat.append(np.var(cf, ddof=1))
            sigma_feat.append(np.log(np.var(cf)))
            skew_feat.append(skew(cf))
            kurt_feat.append(kurtosis(cf))

        sigma_feat_ls.append(sigma_feat)
        skew_feat_ls.append(skew_feat)
        kurt_feat_ls.append(kurt_feat)    

    sigma_feat = np.array(sigma_feat_ls)
    skew_feat = np.array(skew_feat_ls)
    kurt_feat = np.array(kurt_feat_ls)
    
    return sigma_feat, skew_feat, kurt_feat

In [15]:
laser_train_sigma, laser_train_skew, laser_train_kurt = compute_coefficients(train_files_laser)
original_train_sigma, original_train_skew, original_train_kurt = compute_coefficients(train_files_original) 

In [16]:
laser_test_sigma, laser_test_skew, laser_test_kurt = compute_coefficients(test_files_laser)
original_test_sigma, original_test_skew, original_test_kurt = compute_coefficients(test_files_original)

In [17]:
print(laser_train_sigma.shape)
print(original_train_sigma.shape)

(70, 6)
(70, 6)


In [18]:
print(laser_test_sigma.shape)
print(original_test_sigma.shape)

(25, 6)
(25, 6)


## SVM with Linear Kernel

In [19]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [20]:
X_train_laser = np.concatenate((laser_train_sigma, laser_train_skew, laser_train_kurt), axis=1)
train_laser_label = np.array(['laser']*X_train_laser.shape[0])
print(train_laser_label)
print(train_laser_label.shape)
X_train_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser']
(70,)


(70, 18)

In [21]:
X_train_orig = np.concatenate((original_train_sigma, original_train_skew, original_train_kurt), axis=1)
train_orig_label = np.array(['original']*X_train_orig.shape[0])
print(train_orig_label)
print(train_orig_label.shape)
X_train_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original']
(70,)


(70, 18)

In [22]:
X_test_laser = np.concatenate((laser_test_sigma, laser_test_skew, laser_test_kurt), axis=1)
test_laser_label = np.array(['laser']*X_test_laser.shape[0])
print(test_laser_label)
print(test_laser_label.shape)
X_test_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser']
(25,)


(25, 18)

In [23]:
X_test_orig = np.concatenate((original_test_sigma, original_test_skew, original_test_kurt), axis=1)
test_orig_label = np.array(['original']*X_test_orig.shape[0])
print(test_orig_label)
print(test_orig_label.shape)
X_test_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original']
(25,)


(25, 18)

In [24]:
svm_clf = make_pipeline(StandardScaler(), LinearSVC())

In [25]:
X_train = np.concatenate((X_train_laser, X_train_orig))
X_train.shape

(140, 18)

In [26]:
X_test = np.concatenate((X_test_laser, X_test_orig))
X_test.shape

(50, 18)

In [27]:
y_train = np.concatenate((train_laser_label, train_orig_label))
y_train.shape

(140,)

In [28]:
y_test = np.concatenate((test_laser_label, test_orig_label))
y_test.shape

(50,)

In [29]:
svm_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [30]:
y_pred = svm_clf.predict(X_test)

In [31]:
y_pred

array(['laser', 'laser', 'laser', 'laser', 'laser', 'laser', 'laser',
       'laser', 'laser', 'laser', 'laser', 'laser', 'laser', 'laser',
       'laser', 'laser', 'laser', 'laser', 'laser', 'laser', 'original',
       'original', 'laser', 'original', 'original', 'original',
       'original', 'original', 'original', 'original', 'original',
       'original', 'original', 'original', 'original', 'original',
       'original', 'original', 'original', 'original', 'original',
       'original', 'original', 'original', 'original', 'original',
       'original', 'original', 'original', 'original'], dtype='<U8')

In [32]:
print("accuracy = ", accuracy_score(y_test, y_pred))

accuracy =  0.92


In [33]:
cm = confusion_matrix(y_test, y_pred)

In [35]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svm_clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### GMM as classifier

In [152]:
label_map = {'laser': 0, 'original': 1}

In [164]:
y_train_num = np.array([label_map[x] for x in y_train])
y_train_num

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [165]:
y_test_num = np.array([label_map[x] for x in y_test])
y_test_num

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [166]:
from sklearn.mixture import GaussianMixture

In [276]:
gmm_clf = make_pipeline(StandardScaler(), GaussianMixture(n_components=2, random_state=0))
# gmm_clf = GaussianMixture(n_components=2)

In [277]:
# means = []
# for i in range(len(label_map.keys())):
#     print(i)
# #     ids = y_train_num == i
# #     print(ids)
#     m = X_train[y_train_num == i].mean(axis=0)
#     means.append(m)

gmm_clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussianmixture',
                 GaussianMixture(n_components=2, random_state=0))])

In [278]:
gmm_clf.means_init = np.array(
        [X_train[y_train_num == i].mean(axis=0) for i in range(len(label_map.keys()))])

In [285]:
gmm_clf[1].means_

array([[ 0.81747863,  1.14681719,  1.18867321,  0.94945236,  0.54201093,
         0.25068733, -0.05486399, -0.54094801, -0.34840267, -0.74417221,
        -0.61874772, -0.4897246 , -0.59546941, -1.02920677, -1.08196364,
        -1.17372463, -0.95463521, -0.88415007],
       [-0.41313436, -0.57957427, -0.60072732, -0.47983076, -0.2739195 ,
        -0.12669145,  0.02772696,  0.27338232,  0.17607447,  0.37608703,
         0.31270046,  0.24749523,  0.30093615,  0.52013676,  0.54679883,
         0.59317266,  0.48245005,  0.44682853]])

In [280]:
gmm_clf.fit(X_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussianmixture',
                 GaussianMixture(n_components=2, random_state=0))])

In [283]:
gmm_clf[1].converged_

True

In [270]:
y_pred_gmm = gmm_clf.predict(X_test)

AttributeError: 'GaussianMixture' object has no attribute 'means_'

In [259]:
y_pred_gmm

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1])

In [260]:
probs = gmm_clf.predict_proba(X_train)
print(probs.round(3))

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [261]:
print("accuracy = ", accuracy_score(y_test_num, y_pred_gmm))

accuracy =  0.72


In [262]:
y_pred_gmm_label = np.array(['laser' if x == 0 else 'original' for x in y_pred_gmm])

In [263]:
cm = confusion_matrix(y_test, y_pred_gmm_label)

In [264]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …